In [1]:
import numpy as np
import pandas as pd
import re, nltk, spacy, gensim
import requests
from bs4 import BeautifulSoup

# Sklearn
from sklearn.decomposition import LatentDirichletAllocation, TruncatedSVD
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import GridSearchCV
from pprint import pprint

# Plotting tools
import pyLDAvis
import pyLDAvis.sklearn
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
def get_scripts(show,start=0,limit=None):
    url = f'https://www.springfieldspringfield.co.uk/episode_scripts.php?tv-show={show}'
    
    page = requests.get(url)
    print(page)
    bs = BeautifulSoup(page.content)
    
    episode_titles_bs = bs.find_all(attrs={'class':'season-episode-title'})
    
    hrefs = ['https://www.springfieldspringfield.co.uk/'+x['href'] for x in episode_titles_bs]
    
    if limit:
        scripts = []
        failed_hrefs = []
        for url in hrefs[start:start+limit]:
            try:
                page = requests.get(url)
                #print(page)
                bs = BeautifulSoup(page.content)

                scripts.append(bs.find(attrs={'class':'scrolling-script-container'}))
            except:
                failed_hrefs.append(url)
    else:
        scripts = []
        failed_hrefs = []
        for url in hrefs:
            try:
                page = requests.get(url)
                #print(page)
                bs = BeautifulSoup(page.content)

                scripts.append(bs.find(attrs={'class':'scrolling-script-container'}))
            except:
                failed_hrefs.append(url)
        
        
    print(failed_hrefs)
    scripts = [x.get_text() for x in scripts]
    if scripts==None:
        return 'Failure'
    else:
        scripts = [x.replace('\r',' ').replace('\n', ' ').replace('\t', ' ').strip() for x in scripts]
    
    with open(f'{show}.txt', 'w') as filehandle:
        for listitem in scripts:
            filehandle.write('%s\n' % listitem)
    
        return scripts

In [3]:
scripts = get_scripts('cheers')
type(scripts)

<Response [200]>
[]


list

In [4]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

In [5]:
data_words = [x for x in list(sent_to_words(scripts))]

In [13]:
def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append(" ".join([token.lemma_ if token.lemma_ not in ['-PRON-'] else '' for token in doc if token.pos_ in allowed_postags]))
    return texts_out

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# Run in terminal: python3 -m spacy download en
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

# Do lemmatization keeping only Noun, Adj, Verb, Adverb
data_lemmatized = lemmatization(data_words, allowed_postags=['PROPN'])

In [14]:
vectorizer = CountVectorizer(analyzer='word',       
                             min_df=10,                        # minimum reqd occurences of a word 
                             stop_words='english',             # remove stop words
                             lowercase=True,                   # convert all words to lowercase
                             token_pattern='[a-zA-Z0-9]{3,}',  # num chars > 3
                             # max_features=50000,             # max number of uniq words
                            )

data_vectorized = vectorizer.fit_transform(data_lemmatized)

In [15]:
# Define Search Param
search_params = {'n_components': [5, 6, 7], 
                 'learning_decay': [.7, .8, .9],
                 'max_iter':[10,20,30],
                 'n_jobs':[-1],}

# Init the Model
lda = LatentDirichletAllocation()

# Init Grid Search Class
model = GridSearchCV(lda, param_grid=search_params)

# Do the Grid Search
model.fit(data_vectorized)

//anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


GridSearchCV(cv='warn', error_score='raise-deprecating',
             estimator=LatentDirichletAllocation(batch_size=128,
                                                 doc_topic_prior=None,
                                                 evaluate_every=-1,
                                                 learning_decay=0.7,
                                                 learning_method='batch',
                                                 learning_offset=10.0,
                                                 max_doc_update_iter=100,
                                                 max_iter=10,
                                                 mean_change_tol=0.001,
                                                 n_components=10, n_jobs=None,
                                                 perp_tol=0.1,
                                                 random_state=None,
                                                 topic_word_prior=None,
                                   

In [16]:
# Best Model
best_lda_model = model.best_estimator_

# Model Parameters
print("Best Model's Params: ", model.best_params_)

# Log Likelihood Score
print("Best Log Likelihood Score: ", model.best_score_)

# Perplexity
print("Model Perplexity: ", best_lda_model.perplexity(data_vectorized))

Best Model's Params:  {'learning_decay': 0.7, 'max_iter': 30, 'n_components': 5, 'n_jobs': -1}
Best Log Likelihood Score:  -35262.595759908356
Model Perplexity:  40.65193140401971


In [17]:
# Create Document - Topic Matrix
lda_output = best_lda_model.transform(data_vectorized)

# column names
topicnames = ["Topic" + str(i) for i in range(best_lda_model.n_components)]

# index names
docnames = ["Doc" + str(i) for i in range(len(scripts))]

# Make the pandas dataframe
df_document_topic = pd.DataFrame(np.round(lda_output, 2), columns=topicnames, index=docnames)

# Get dominant topic for each document
dominant_topic = np.argmax(df_document_topic.values, axis=1)
df_document_topic['dominant_topic'] = dominant_topic

# Styling
def color_green(val):
    color = 'green' if val > .1 else 'black'
    return 'color: {col}'.format(col=color)

def make_bold(val):
    weight = 700 if val > .1 else 400
    return 'font-weight: {weight}'.format(weight=weight)

# Apply Style
df_document_topics = df_document_topic.head(15).style.applymap(color_green).applymap(make_bold)
df_document_topics

,Topic0,Topic1,Topic2,Topic3,Topic4,dominant_topic
Doc0,0,0,0,0.99,0,3
Doc1,0,0,0,0.99,0,3
Doc2,0,0,0,0.99,0,3
Doc3,0,0,0.36,0.63,0,3
Doc4,0,0,0,0.99,0,3
Doc5,0,0.41,0,0.58,0,3
Doc6,0,0,0.18,0.25,0.57,4
Doc7,0,0,0.39,0.61,0,3
Doc8,0,0,0.08,0.91,0,3
Doc9,0,0,0,0.99,0,3


In [18]:
df_topic_distribution = df_document_topic['dominant_topic'].value_counts().reset_index(name="Num Documents")
df_topic_distribution.columns = ['Topic Num', 'Num Documents']
df_topic_distribution

,Topic Num,Num Documents
0,3,145
1,1,56
2,4,24
3,0,24
4,2,15


In [19]:
pyLDAvis.enable_notebook()
panel = pyLDAvis.sklearn.prepare(best_lda_model, data_vectorized, vectorizer, mds='tsne')
panel


//anaconda3/lib/python3.7/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=               x           y  topics  cluster       Freq
topic                                                   
3      -2.222704   74.184837       1        1  50.626916
1     -77.960823  -15.629301       2        1  23.601922
0     -83.558762 -132.980301       3        1  10.904411
2      37.648018 -123.948174       4        1   7.559625
4      80.455154  -14.904718       5        1   7.307126, topic_info=    Category         Freq     Term        Total  loglift  logprob
20   Default  1309.000000    carla  1309.000000  30.0000  30.0000
46   Default  4440.000000      don  4440.000000  29.0000  29.0000
49   Default   302.000000    eddie   302.000000  28.0000  28.0000
165  Default  4228.000000      sam  4228.000000  27.0000  27.0000
159  Default   564.000000  rebecca   564.000000  26.0000  26.0000
..       ...          ...      ...          ...      ...      ...
166   Topic5    36.398654    sammy   941.041793  -0.6361  -3.9358
70    Topic5    27.093194      god   514.673966  -0.3279  -4.2311
195   Topic5    24.743532    wanna   593.969560  -0.5619  -4.3218
45    Topic5    14.721442    doesn   142.815284   0.3441  -4.8410
12    Topic5    13.773858   boston   256.745437  -0.3090  -4.9076

[275 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
0         5  0.994902       aii
1         1  0.323163   america
1         2  0.399202   america
1         3  0.076038   america
1         4  0.152077   america
...     ...       ...       ...
209       2  0.755635       yay
210       1  0.636841      york
210       3  0.106140      york
210       4  0.265350      york
211       5  0.948066  yourseif

[466 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[4, 2, 1, 3, 5])